In [ ]:
#%matplotlib inline  
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import csv
import glob
import os.path
import powerlaw
from collections import defaultdict

def discrete_cmap(N, base_cmap=None):
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

def cm2inch(value):
    return value/2.54

x = []
y = []
z = []
datafile = '../DATA/distribution_compare/all_merged.txt'
figoutfile = './FIG001_BESTFIT_ALL.pdf'
count = defaultdict(float)
with open(datafile, 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter = '\t')
    for row in reader:
        if(len(row) == 0):
            continue
        if(unicode(row[0]).isnumeric()):
            x.append(int(row[0]))
            y.append(int(row[1]))
            z.append(float(row[3]))
            count[int(row[0])] = float(row[2])

print "Average published article per year for all articles:", sum(count.values())/len(count.values())


nx = max(x) - min(x) + 1
ny = max(y) - min(y) + 1

Z = []

for i in range(nx):
    temparray = []
    for j in range(ny):
        temparray.append(0)
    Z.append(temparray)

for i in range(len(x)):
    Z[x[i]-min(x)][y[i]-min(y)] = z[i] 

fig = plt.figure(figsize =(cm2inch(19)*2,cm2inch(5)*2))

ax1 = fig.add_subplot(131)
ax1.set_xlabel('Cited year')
ax1.set_ylabel('Published year')
     
cmap=discrete_cmap(6, 'nipy_spectral')
cmap.set_under(color='white') 
plt.pcolor(np.arange(nx),np.arange(ny),Z, cmap = cmap)

labels = ['PL', 'PLE', 'LN', 'LNP', 'EXP', 'StEX']        
cbar = plt.colorbar()
cbar.set_ticks([0,1,2,3,4,5])
cbar.set_ticklabels(labels)

plt.xlim(0,nx - 1)
plt.ylim(0,ny - 1)

zmin = min(i for i in z if i > 0)

xlabels = [i for i in range(min(x), max(x)+1, 4)]
ylabels = [i for i in range(min(y), max(y)+1, 4)]
plt.xticks([i for i in range(0, nx - 1 , 4)],xlabels)
plt.yticks([i for i in range(0, ny - 1 , 4)],ylabels)

plt.clim(-0.5,5.5)
x = []
y = []
z = []
datafile = '../DATA/power_law_exp_exponents/all_merged.txt'

count = defaultdict(float)
with open(datafile, 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter = '\t')
    for row in reader:
        if(len(row) == 0):
            continue
        if(unicode(row[0]).isnumeric()):
            x.append(int(row[0]))
            y.append(int(row[1]))
            z.append(float(row[3]))
            count[int(row[0])] = float(row[2])

nx = max(x) - min(x) + 1
ny = max(y) - min(y) + 1

Z = []

for i in range(nx):
    temparray = []
    for j in range(ny):
        temparray.append(0)
    Z.append(temparray)

for i in range(len(x)):
    Z[x[i]-min(x)][y[i]-min(y)] = z[i] 

ax3 = fig.add_subplot(132)

ax3.set_xlabel('Cited year')
ax3.set_ylabel('Published year')

cmap=plt.cm.jet
cmap.set_under(color='white') 

try:   
    plt.pcolor(np.arange(nx),np.arange(ny),Z, cmap = cmap, vmin = 0.00001)#, norm=colors.LogNorm())
    plt.colorbar()

except:
    print "ERROR OCCURS"

plt.xlim(0,nx - 1)
plt.ylim(0,ny - 1)

zmin = min(i for i in z if i > 0)
zmax = max(i for i in z if i > 0)

xlabels = [i for i in range(min(x), max(x)+1, 4)]
ylabels = [i for i in range(min(y), max(y)+1, 4)]
plt.xticks([i for i in range(0, nx - 1 , 4)],xlabels)
plt.yticks([i for i in range(0, ny - 1 , 4)],ylabels)

plt.clim(1.7,4.8)

x = []
y = []
z = []

f_citecount = sorted(glob.glob("../DATA/citecounts/ALL_*.txt"))

ax5 = fig.add_subplot(133)
ax5.set_xlabel('Yearly acquired number of citations')
ax5.set_ylabel('Probability density')
ax5.set_ylim([2e-10, 2])
m = plt.cm.ScalarMappable(cmap=plt.cm.jet)
m.set_array([0,17])    
for datafile in f_citecount:
    data = np.loadtxt(datafile)
    elems = datafile.split("/")[-1].split("_")
    try:
        diff = int(elems[2].split(".")[0]) - int(elems[1]) 
    except:
        print elems
    if(len(data[data>0]) == 0):
        continue
    try:
        powerlaw.plot_pdf(data[data>0], ax5, color=plt.cm.jet(float(diff)/17))
    except IndexError as e:
        continue

plt.colorbar(m).set_label('Cited year - Published year')

labellist = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m"]
index = 0
for ax in fig.get_axes():
    if(index%2 == 0):
        ax.yaxis.label.set_fontsize(14)
        ax.xaxis.label.set_fontsize(14)       
        ax.tick_params(axis='x', labelsize=14)
        ax.tick_params(axis='y', labelsize=14)       
        ax.text(-0.2, 1, labellist[index/2], fontsize=14, weight='bold', transform=ax.transAxes)  
    index += 1

plt.tight_layout()
plt.savefig(figoutfile)
plt.show()
plt.close()